# Gesture Recognition
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started.

In [1]:
import numpy as np
import os
from scipy.misc import imread, imresize
import datetime
import os

ImportError: cannot import name 'imread' from 'scipy.misc' (C:\ProgramData\Anaconda3\lib\site-packages\scipy\misc\__init__.py)

We set the random seed so that the results don't vary drastically.

In [ ]:
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf
tf.set_random_seed(30)

In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

In [ ]:
train_doc = np.random.permutation(open('./Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('./Project_data/val.csv').readlines())
batch_size =64#experiment with the batch size
input_shape =(15,100,100,3)

## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [ ]:
sample_image = os.listdir('./Project_data/train'+'/'+ np.random.permutation(train_doc)[0 + (0)].split(';')[0])

In [ ]:
print(sample_image)

In [ ]:
sample_im_path = './Project_data/train'+'/'+ train_doc[0].split(';')[0]
sample = imread(sample_im_path+'/'+os.listdir(sample_im_path)[0])

In [ ]:
import cv2
import matplotlib.pyplot as plt
% matplotlib inline


In [ ]:
plt.imshow(sample)

In [ ]:
sample.shape

In [ ]:
sample = imresize(sample,(100,100))

In [ ]:
plt.imshow(sample)

In [ ]:
def generator(source_path, folder_list, batch_size):
        
        print( 'Source path = ', source_path, '; batch size =', batch_size)
        
        img_idx = np.arange(0,30,2)#create a list of image numbers you want to use for a particular video
        
        while True:
            t = np.random.permutation(folder_list)
            num_batches = len(folder_list)//batch_size # calculate the number of batches
            for batch in range(num_batches): # we iterate over the number of batches
                batch_data = np.zeros((batch_size,len(img_idx),100,100,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
                batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
                for folder in range(batch_size): # iterate over the batch_size
                    imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                    for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                        image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)

                        #crop the images and resize them. Note that the images are of 2 different shape 
                        #and the conv3D will throw error if the inputs in a batch have different shapes
                        
                        image = imresize(image,(100,100))
                        
                        
                        batch_data[folder,idx,:,:,0] =image[:,:,0]/255#normalise and feed in the image
                        batch_data[folder,idx,:,:,1] =image[:,:,1]/255#normalise and feed in the image
                        batch_data[folder,idx,:,:,2] =image[:,:,2]/255#normalise and feed in the image

                    batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
                yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do
            
            rem_image = len(folder_list)%batch_size
            batch += 1
            if(rem_image!=0):
                batch_data = np.zeros((rem_image,len(img_idx),100,100,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
                batch_labels = np.zeros((rem_image,5)) # batch_labels is the one hot representation of the output
                for folder in range(rem_image): # iterate over the batch_size
                    imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                    for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                        image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)

                        #crop the images and resize them. Note that the images are of 2 different shape 
                        #and the conv3D will throw error if the inputs in a batch have different shapes

                        image = imresize(image,(100,100))
                        image = image/255
                        batch_data[folder,idx,:,:,0] = image[:,:,0]
                        batch_data[folder,idx,:,:,1] = image[:,:,1]
                        batch_data[folder,idx,:,:,2] = image[:,:,2]
                    batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels

Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [ ]:

train_path = './Project_data/train'
val_path = './Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs =50 # choose the number of epochs
print ('# epochs =', num_epochs)
num_classes = 5
print ('# number of classes =', num_classes)

## Model
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

In [ ]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation,Dropout
from keras.layers.convolutional import  Conv2D, MaxPooling2D, AveragePooling2D,Conv3D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers
from keras.regularizers import l2
from keras.layers import LSTM, GRU, Bidirectional, SimpleRNN, RNN




#write your model here

class ModelBuilder(object):
    
   
    
    # Convolution 3D Models - Conv3D and MaxPooling3D 

    
    @staticmethod
    def conv3d_model_1(input_shape, num_classes, dropout = 0.50):

        model = Sequential()
        
        model.add(Conv3D(16, (3,3,3), padding='same',input_shape = (15,100,100,3)))
        model.add(Activation('relu'))
        model.add(BatchNormalization())

        model.add(Conv3D(32, (3,3,3), padding='same'))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(MaxPooling3D(pool_size=(2, 2, 2)))

        model.add(Conv3D(64, (3,3,3), padding='same'))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(MaxPooling3D(pool_size=(2, 2, 2)))

        model.add(Conv3D(128, (3,3,3), padding='same'))
        model.add(Activation('relu'))
        model.add(BatchNormalization())
        model.add(MaxPooling3D(pool_size=(2, 2, 2)))

        model.add(Flatten())
        model.add(Dense(128, activation='relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.5))

        model.add(Dense(64,activation='relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.5))

        model.add(Dense(5,activation='softmax'))

        return model
    
    
       
   
    
   

Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [ ]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)


In [ ]:
class ModelTrainer:
    
    ###############################################
    # Method to initialize the model train params #
    ###############################################
    @staticmethod
    def InitializeModelParams():
        
        # Model train parameters
        num_train_sequences = len(train_doc)
        print('# training sequences =', num_train_sequences)
        
        num_val_sequences = len(val_doc)
        print('# validation sequences =', num_val_sequences)
        
        print('# batch size =', batch_size)    
        print('# epochs =', num_epochs)
        
        
        
    ##########################################################################
    # Model Train                                                            #
    ##########################################################################
    @staticmethod
    def train(model, batch_size, num_epochs, train_generator, val_generator, optimiser = None):

         # Initialize the model optimizer
        if optimiser == None:
        #optimiser ='sgd' #write your optimizer
           optimiser=optimizers.Adam()#(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
        # compile it
        model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

        # summary of model
        print (model.summary())
    
    
    
    ####################################################
        curr_dt_time = datetime.datetime.now()
        model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'

        if not os.path.exists(model_name):
            os.mkdir(model_name)

        filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

    # generate the model checkpoint.
        checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)
    
    # write the REducelronplateau code here
        LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2, cooldown=1, verbose=1)
        callbacks_list = [checkpoint, LR]

    #################################################################
        if (num_train_sequences%batch_size) == 0:
            steps_per_epoch = int(num_train_sequences/batch_size)
        else:
            steps_per_epoch = (num_train_sequences//batch_size) + 1

        if (num_val_sequences%batch_size) == 0:
            validation_steps = int(num_val_sequences/batch_size)
        else:
            validation_steps = (num_val_sequences//batch_size) + 1

#####################################################################################

        model_history=model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1,callbacks=callbacks_list, validation_data=val_generator,validation_steps=validation_steps, class_weight=None,workers=1, initial_epoch=0)
        
        return model_history



     ##########################################################################
    # This method takes the model train history as parameter and plots the   #
    # Loss & Accuracy curves for the train and validation data.              #
    ##########################################################################
   
    @staticmethod
    def plot_model_history(history):
        fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(15,4))
        axes[0].plot(history.history['loss'])   
        axes[0].plot(history.history['val_loss'])
        axes[0].legend(['loss','val_loss'])

        axes[1].plot(history.history['categorical_accuracy'])   
        axes[1].plot(history.history['val_categorical_accuracy'])
        axes[1].legend(['categorical_accuracy','val_categorical_accuracy'])


The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

# MODEL 

### Conv3D - MaxPooling3D

In [ ]:
# build the model
conv3d_1 = ModelBuilder.conv3d_model_1(input_shape, num_classes)
 


In [ ]:
# train the model
model_history_conv3d_1 = ModelTrainer.train(conv3d_1, batch_size, num_epochs, train_generator, val_generator)


    

In [ ]:
# plot the model train and validation history - loss and accuracy curves
ModelTrainer.plot_model_history(model_history_conv3d_1)